<img src="header.png" align="left"/>

# Anwendungsbeispiel Import of audio data with classification

Das Ziel dieses Beispieles ist es die Arbeit mit Audiodaten, den Import, die Vorbereitung und die Klassifikation zu erklären. Dabei werden folgende Schritte durchgeführt:

- Dynamisches Laden und entpacken der Audiodaten von einer externen Quelle
- Review der Organisation auf dem Filesystem
- Laden der Daten
- Transformationen
- Training
- Analyse

Der verwendete Datensatz heisst ESC-50 [1] mit 50 Klassen von Geräuschen in Dateien organisiert. Die Audiodaten sind jeweils 5 Sekunden lang und haben 40 Samples pro Klasse.


Der Code für das Beispiel is aus [2] und [3] kombiniert.


Quellen für die Beispiele und Daten:

- [1] [https://github.com/karolpiczak/ESC-50/blob/master/LICENSE](https://github.com/karolpiczak/ESC-50/blob/master/LICENSE) (Hinweise auf Unterlizenzen der Daten)
- [2] [https://github.com/CarmineCella/esc50_keras/blob/master/esc50_keras.py](https://github.com/CarmineCella/esc50_keras/blob/master/esc50_keras.py)
- [3] [https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7](https://medium.com/@mikesmales/sound-classification-using-deep-learning-8bc2aa1990b7)

Zitat der Datenquelle:
```
K. J. Piczak. ESC: Dataset for Environmental Sound Classification. Proceedings of the 23rd Annual ACM Conference on Multimedia, Brisbane, Australia, 2015.
```





In [143]:
import os
import fnmatch
import joblib
import librosa
import numpy as np
import os.path
import zipfile
from urllib.request import urlretrieve

import pandas as pd

from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.callbacks import History
from keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping

from keras.utils import np_utils
from sklearn.svm import SVC
from keras.preprocessing.image import ImageDataGenerator
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt


#
# Abdrehen von Fehlermeldungen
#
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)
simplefilter(action='ignore', category=Warning)


In [122]:
#
# Für GPU Support
#
import tensorflow as tf
print ( tf.__version__ ) 

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR )
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))


2.0.0


# Hilfsfunktionen

In [123]:
urlDataSource = 'https://github.com/karoldvl/ESC-50/archive/master.zip'
localExtractionFolder = 'data/ESC-50'
localDataArchive = 'data/ESC-50/master.zip'

sampleLen = 110250 # in samples is 5 sec @ 22050


In [124]:
#
# Laden der Daten von einer URL
#
def download_dataset(url,dataset_file_path):
    if os.path.exists(localDataArchive):
        print("archive already downloaded.")
    else:
        print("started loading archive from url {}".format(url))
        filename, headers = urlretrieve(url, dataset_file_path)
        print("finished loading archive from url {}".format(url))

def extract_dataset(dataset_file_path, extraction_directory):    
    if (not os.path.exists(extraction_directory)):
        os.makedirs(extraction_directory)        
    zip = zipfile.ZipFile(dataset_file_path)
    zip.extractall(path=extraction_directory)        
    print("extraction of dataset from {} to {} done.".format(dataset_file_path,extraction_directory) )


# Laden der Daten

In [125]:
#
# Laden der Daten ausführen
#
download_dataset(urlDataSource,localDataArchive)

archive already downloaded.


In [126]:
#
# Extrahieren der Daten
#
extract_dataset(localDataArchive,localExtractionFolder)

extraction of dataset from data/ESC-50/master.zip to data/ESC-50 done.


# Organisation von Audiodaten auf dem Filesystem

Die Audiodateien liegen alle in einem Verzeichnis. Die Zuordnung der Klasse ist im Dateinamen kodiert und in einer Datenbank gespeichert.
Details dazu unter [[1]](https://github.com/karolpiczak/ESC-50)


In [127]:
#
# Auslesen der Datenbank
#
df = pd.read_csv( localExtractionFolder + '/ESC-50-master/meta/esc50.csv')

df.head()

,filename,fold,target,category,esc10,src_file,take
0,1-100032-A-0.wav,1,0,dog,True,100032,A
1,1-100038-A-14.wav,1,14,chirping_birds,False,100038,A
2,1-100210-A-36.wav,1,36,vacuum_cleaner,False,100210,A
3,1-100210-B-36.wav,1,36,vacuum_cleaner,False,100210,B
4,1-101296-A-19.wav,1,19,thunderstorm,False,101296,A


# Analyse der Daten

In [ ]:
#
# Anzeige als PCM Kurven
#

# Erzeugen der Trainingsdaten

In [128]:
def get_features (file, hop, bins):
    
    y = np.zeros(sampleLen);   
    yt, sr = librosa.core.load  (file, mono=True)
    
    if len(yt) == 0: 
        print ('found empty file ' + file )
        return 0

    min_length = min(len(y), len(yt))
    y[:min_length] = yt[:min_length]
    
    C = librosa.feature.mfcc(y=y, sr=sr, hop_length=hop, n_mfcc = bins)  
    #print('mfcc {}'.format(C) )
        
    return C

memory = joblib.Memory(cachedir=localExtractionFolder+'/esc50_joblib', verbose=0)
cached_get_features = memory.cache(get_features)




def compute_features (root_path):
    
    hop = 2048
    bins = 100
        
    classes = 50
    samples = 0

    y_data = []    
    X_data = []
    
    for root, dir, files in os.walk(root_path):
        
        waves = fnmatch.filter(files, "*.wav")

        if len(waves) != 0:
            for item in waves:
                # e.g. 2-39443-A-19.wav
                fileName = os.path.splitext ( os.path.basename(item) )[0] 
                classID = int(fileName.split('-')[3])
                
                mfcc = cached_get_features( os.path.join(root, item), hop, bins)
                print(".",end='')

                X_data.append(mfcc)
                y_data.append(classID)
                
                samples = samples + 1
                if samples >= 1000:
                    break
    
    X_data = np.stack(X_data, axis=2)
    X_data = np.transpose(X_data, (2,0,1))
    d1 = X_data.shape[0]
    d2 = X_data.shape[1]
    d3 = X_data.shape[2]    
    X_data = np.reshape(X_data, (d1,d2,d3,1))
    y_data = np.array(y_data)    

    print ("samples = " + str (samples))

    return X_data, y_data, classes, samples

In [129]:
x_data, y_data, classes, samples = compute_features ("data/ESC-50/ESC-50-master/audio")

(100, 54, 1000)
(1000, 100, 54)
(1000, 100, 54, 1)
(1000,)
classes = 50
samples = 1000


In [149]:
x_data = x_data.astype('float32')
y_data = y_data.astype('uint8')

# Anzeige als Bild

In [131]:
def standardize (x):
    mu = np.mean (x, axis=0)
    de = np.std (x, axis=0)
    
    eps = np.finfo('float32').eps
    x = (x - mu) / (eps + de)
    return x


In [132]:
x_data = standardize(x_data)

In [133]:
# convert class vectors to binary class matrices
y_data = np_utils.to_categorical(y_data, classes )

# Train und Test Split

In [134]:
#
# Split der Daten in Train und Test(validation) Datensätze
#
x_train, x_validation, y_train, y_validation = train_test_split(x_data, y_data, test_size=0.25, random_state=42)

print('shapes {} {} {} {}'.format(x_train.shape, x_validation.shape, y_train.shape, y_validation.shape ))

shapes (750, 100, 54, 1) (250, 100, 54, 1) (750, 50) (250, 50)


# Model bauen

In [135]:
#
# Erzeugen eines einfache Modelles
#
def createModel():
    
    img_rows = x_train.shape[1]
    img_cols = x_train.shape[2]
    
    model = Sequential()
    model.add(Convolution2D(32, kernel_size=5, strides=1, border_mode='same', input_shape=(img_rows, img_cols, 1)))
    model.add(Activation('relu'))
    model.add(Convolution2D(32, kernel_size=3, strides=1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
     
    model.add(Convolution2D(64, kernel_size=3, strides=1, border_mode='same'))
    model.add(Activation('relu'))
    model.add(Convolution2D(64, kernel_size=3, strides=1))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model


In [136]:
model_cnn = createModel()
model_cnn.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 100, 54, 32)       832       
_________________________________________________________________
activation_26 (Activation)   (None, 100, 54, 32)       0         
_________________________________________________________________
conv2d_24 (Conv2D)           (None, 98, 52, 32)        9248      
_________________________________________________________________
activation_27 (Activation)   (None, 98, 52, 32)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 49, 26, 32)        0         
_________________________________________________________________
dropout_12 (Dropout)         (None, 49, 26, 32)        0         
_________________________________________________________________
conv2d_25 (Conv2D)           (None, 49, 26, 64)       

In [144]:
#optimizer = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
optimizer = Adam ()
model_cnn.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [145]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.4,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.4,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [146]:
datagen.fit(x_train)

In [147]:
callbacks = [EarlyStopping(monitor='val_loss', patience=4, verbose=1, mode='auto'), History()]

In [148]:
model_cnn.fit_generator(datagen.flow(x_train, y_train,
                            batch_size=32),
                            samples_per_epoch=x_train.shape[0],
                            nb_epoch=20,
                            callbacks=callbacks,
                            validation_data=(x_validation, y_validation))

Epoch 1/20
23/23 [==============================] - 16s 688ms/step - loss: 3.7168 - accuracy: 0.0613 - val_loss: 3.7558 - val_accuracy: 0.0560
Epoch 2/20
23/23 [==============================] - 16s 676ms/step - loss: 3.6143 - accuracy: 0.0780 - val_loss: 3.6265 - val_accuracy: 0.0600
Epoch 3/20
23/23 [==============================] - 16s 688ms/step - loss: 3.5018 - accuracy: 0.0965 - val_loss: 3.5193 - val_accuracy: 0.0920
Epoch 4/20
23/23 [==============================] - 15s 631ms/step - loss: 3.3855 - accuracy: 0.1171 - val_loss: 3.5685 - val_accuracy: 0.0920
Epoch 5/20
23/23 [==============================] - 15s 632ms/step - loss: 3.3303 - accuracy: 0.1295 - val_loss: 3.4797 - val_accuracy: 0.1000
Epoch 6/20
23/23 [==============================] - 15s 633ms/step - loss: 3.2970 - accuracy: 0.1518 - val_loss: 3.3003 - val_accuracy: 0.1240
Epoch 7/20
23/23 [==============================] - 14s 601ms/step - loss: 3.2562 - accuracy: 0.1295 - val_loss: 3.3129 - val_accuracy: 0.1000